<center><h1>ResNet para Classificação das Imagens GASF-GADF-MTF</h1></center>

In [1]:
from __future__ import print_function  
from keras.models import Model
from keras.layers import Input, Dense, merge, Activation
from keras.layers.merge import Concatenate
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras
import time
import pickle
from keras.callbacks import ReduceLROnPlateau
np.random.seed(7)

Using TensorFlow backend.


<font color="red"><b><h3>Configurações</h3></b></font>

In [2]:
num_epochs = 1000
datafiles  = ['imagens_Coffee_16x16.pkl',
              'imagens_OliveOil_16x16.pkl',
              'imagens_Beef_16x16.pkl']

<font color="Blue"><b><h3>Estrutura da ResNet</h3></b></font>

In [3]:
def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('Construindo ResNet...')
    x = Input(shape=(input_shape))
    conv_x = keras.layers.normalization.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, (8,1), padding='same')(conv_x)
    conv_x = keras.layers.normalization.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)     
    conv_y = keras.layers.Conv2D(n_feature_maps, (5,1), padding='same')(conv_x)
    conv_y = keras.layers.normalization.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)     
    conv_z = keras.layers.Conv2D(n_feature_maps, (3,1), padding='same')(conv_y)
    conv_z = keras.layers.normalization.BatchNormalization()(conv_z)     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, (1,1), padding='same')(x)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.normalization.BatchNormalization()(x)
    y = Concatenate()([shortcut_y, conv_z]);
    y = Activation('relu')(y)
    
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, (8,1), padding='same')(x1)
    conv_x = keras.layers.normalization.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)     
    conv_y = keras.layers.Conv2D(n_feature_maps*2, (5,1), padding='same')(conv_x)
    conv_y = keras.layers.normalization.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)     
    conv_z = keras.layers.Conv2D(n_feature_maps*2, (3,1), padding='same')(conv_y)
    conv_z = keras.layers.normalization.BatchNormalization()(conv_z)     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, (1,1), padding='same')(x1)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.normalization.BatchNormalization()(x1)        
    y = Concatenate()([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, (8,1), padding='same')(x1)
    conv_x = keras.layers.normalization.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)     
    conv_y = keras.layers.Conv2D(n_feature_maps*2, (5,1), padding='same')(conv_x)
    conv_y = keras.layers.normalization.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)     
    conv_z = keras.layers.Conv2D(n_feature_maps*2, (3,1), padding='same')(conv_y)
    conv_z = keras.layers.normalization.BatchNormalization()(conv_z)
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, (1,1), padding='same')(x1)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.normalization.BatchNormalization()(x1)
    y = Concatenate()([shortcut_y, conv_z])
    y = Activation('relu')(y)     
    full = keras.layers.pooling.GlobalAveragePooling2D()(y)   
    out = Dense(nb_classes, activation='softmax')(full)
    print ('Modelo elaborado.')
    return x, out

<font color="Blue"><b><h3>Leitura dos arquivos .pkl gerados pelo Imaging.py</h3></b></font>

In [4]:
def unpickleimg(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    y_train = dict[0][1]
    x_train = dict[0][0]
    y_test = dict[1][1]
    x_test = dict[1][0]
    return y_train, x_train, y_test, x_test

<font color="Green"><b><h3>Execução da Resnet</h3></b></font><br/>
<b>Os resultados serão salvos em arquivos .pkl para cada datafile</b>

In [5]:
for datafile in datafiles:
    y_train, x_train, y_test, x_test = unpickleimg(datafile);
    
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0], 16)     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)     
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)      
    x_test = (x_test - x_train_mean)/(x_train_std) 
     
    x , y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = Model(outputs=y, inputs=x)
    start = time.time()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])      
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,patience=50, min_lr=0.0001) 
    hist = model.fit(x_train,Y_train,batch_size=batch_size,epochs=num_epochs,verbose=1,validation_data=(x_test, Y_test),callbacks = [reduce_lr])

    log = pd.DataFrame(hist.history);
    log['time'] =(time.time() - start);
    print('----------- Resultados -----------');
    print('Loss: ', log.loc[log['loss'].idxmin]['loss'])
    print('Acc: ', log.loc[log['loss'].idxmin]['val_acc']);
    print('Tempo de processamento: %.2f segundos.' % (time.time() - start))
    log.to_pickle('Resultados_'+datafile);


Construindo ResNet...
Modelo elaborado.
Train on 28 samples, validate on 28 samples
Epoch 1/1000
28/28 [==============================] - 3s - loss: 0.5538 - acc: 0.6786 - val_loss: 0.6973 - val_acc: 0.4643
Epoch 2/1000
28/28 [==============================] - 0s - loss: 0.2357 - acc: 0.9643 - val_loss: 0.7007 - val_acc: 0.4643
Epoch 3/1000
28/28 [==============================] - 0s - loss: 0.1427 - acc: 0.9643 - val_loss: 0.7042 - val_acc: 0.4643
Epoch 4/1000
28/28 [==============================] - 0s - loss: 0.0603 - acc: 0.9643 - val_loss: 0.7135 - val_acc: 0.4643
Epoch 5/1000
28/28 [==============================] - 0s - loss: 0.0292 - acc: 1.0000 - val_loss: 0.7286 - val_acc: 0.4643
Epoch 6/1000
28/28 [==============================] - 0s - loss: 0.0157 - acc: 1.0000 - val_loss: 0.7504 - val_acc: 0.4643
Epoch 7/1000
28/28 [==============================] - 0s - loss: 0.0114 - acc: 1.0000 - val_loss: 0.7371 - val_acc: 0.4643
Epoch 8/1000
28/28 [==============================] - 0

28/28 [==============================] - 0s - loss: 0.0010 - acc: 1.0000 - val_loss: 7.4171 - val_acc: 0.5357
Epoch 66/1000
28/28 [==============================] - 0s - loss: 0.0014 - acc: 1.0000 - val_loss: 7.3658 - val_acc: 0.5357
Epoch 67/1000
28/28 [==============================] - 0s - loss: 0.0302 - acc: 1.0000 - val_loss: 7.4690 - val_acc: 0.5357
Epoch 68/1000
28/28 [==============================] - 0s - loss: 0.0028 - acc: 1.0000 - val_loss: 7.4834 - val_acc: 0.5357
Epoch 69/1000
28/28 [==============================] - 0s - loss: 0.1215 - acc: 0.9643 - val_loss: 7.2615 - val_acc: 0.5357
Epoch 70/1000
28/28 [==============================] - 0s - loss: 0.0559 - acc: 1.0000 - val_loss: 4.2141 - val_acc: 0.5357
Epoch 71/1000
28/28 [==============================] - 0s - loss: 0.0358 - acc: 1.0000 - val_loss: 1.9023 - val_acc: 0.5357
Epoch 72/1000
28/28 [==============================] - 0s - loss: 0.0058 - acc: 1.0000 - val_loss: 1.7544 - val_acc: 0.5357
Epoch 73/1000
28/28 [=

28/28 [==============================] - 0s - loss: 4.1230e-05 - acc: 1.0000 - val_loss: 0.6070 - val_acc: 0.7857
Epoch 130/1000
28/28 [==============================] - 0s - loss: 1.6550e-04 - acc: 1.0000 - val_loss: 0.6301 - val_acc: 0.7857
Epoch 131/1000
28/28 [==============================] - 0s - loss: 4.1060e-05 - acc: 1.0000 - val_loss: 0.6478 - val_acc: 0.7857
Epoch 132/1000
28/28 [==============================] - 0s - loss: 4.6490e-04 - acc: 1.0000 - val_loss: 0.6469 - val_acc: 0.7857
Epoch 133/1000
28/28 [==============================] - 0s - loss: 0.0016 - acc: 1.0000 - val_loss: 0.6566 - val_acc: 0.7857
Epoch 134/1000
28/28 [==============================] - 0s - loss: 9.4650e-05 - acc: 1.0000 - val_loss: 0.6469 - val_acc: 0.7857
Epoch 135/1000
28/28 [==============================] - 0s - loss: 4.1017e-05 - acc: 1.0000 - val_loss: 0.6361 - val_acc: 0.7857
Epoch 136/1000
28/28 [==============================] - 0s - loss: 3.6536e-05 - acc: 1.0000 - val_loss: 0.6231 - val

28/28 [==============================] - 0s - loss: 8.5027e-05 - acc: 1.0000 - val_loss: 0.0178 - val_acc: 1.0000
Epoch 193/1000
28/28 [==============================] - 0s - loss: 2.6101e-05 - acc: 1.0000 - val_loss: 0.0163 - val_acc: 1.0000
Epoch 194/1000
28/28 [==============================] - 0s - loss: 2.8136e-04 - acc: 1.0000 - val_loss: 0.0152 - val_acc: 1.0000
Epoch 195/1000
28/28 [==============================] - 0s - loss: 5.1713e-04 - acc: 1.0000 - val_loss: 0.0143 - val_acc: 1.0000
Epoch 196/1000
28/28 [==============================] - 0s - loss: 2.3713e-05 - acc: 1.0000 - val_loss: 0.0134 - val_acc: 1.0000
Epoch 197/1000
28/28 [==============================] - 0s - loss: 0.0679 - acc: 0.9643 - val_loss: 0.0189 - val_acc: 1.0000
Epoch 198/1000
28/28 [==============================] - 0s - loss: 0.0043 - acc: 1.0000 - val_loss: 0.0731 - val_acc: 0.9286
Epoch 199/1000
28/28 [==============================] - 0s - loss: 0.0596 - acc: 0.9643 - val_loss: 0.1395 - val_acc: 0.

28/28 [==============================] - 0s - loss: 8.5059e-05 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 257/1000
28/28 [==============================] - 0s - loss: 7.1168e-04 - acc: 1.0000 - val_loss: 0.0038 - val_acc: 1.0000
Epoch 258/1000
28/28 [==============================] - 0s - loss: 9.8372e-05 - acc: 1.0000 - val_loss: 0.0041 - val_acc: 1.0000
Epoch 259/1000
28/28 [==============================] - 0s - loss: 4.3474e-05 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 260/1000
28/28 [==============================] - 0s - loss: 4.3882e-05 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 1.0000
Epoch 261/1000
28/28 [==============================] - 0s - loss: 1.1782e-04 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 262/1000
28/28 [==============================] - 0s - loss: 8.1806e-05 - acc: 1.0000 - val_loss: 0.0051 - val_acc: 1.0000
Epoch 263/1000
28/28 [==============================] - 0s - loss: 6.8878e-04 - acc: 1.0000 - val_loss: 0.0055 -

...

30/30 [==============================] - 0s - loss: 2.9447e-04 - acc: 1.0000 - val_loss: 2.6033 - val_acc: 0.5333
Epoch 749/1000
30/30 [==============================] - 0s - loss: 2.2141e-04 - acc: 1.0000 - val_loss: 2.6038 - val_acc: 0.5333
Epoch 750/1000
30/30 [==============================] - 0s - loss: 8.9711e-05 - acc: 1.0000 - val_loss: 2.6035 - val_acc: 0.5333
Epoch 751/1000
30/30 [==============================] - 0s - loss: 2.0001e-04 - acc: 1.0000 - val_loss: 2.6038 - val_acc: 0.5333
Epoch 752/1000
30/30 [==============================] - 0s - loss: 8.9690e-04 - acc: 1.0000 - val_loss: 2.6055 - val_acc: 0.5333
Epoch 753/1000
30/30 [==============================] - 0s - loss: 2.4961e-04 - acc: 1.0000 - val_loss: 2.6055 - val_acc: 0.5333
Epoch 754/1000
30/30 [==============================] - 0s - loss: 1.5066e-04 - acc: 1.0000 - val_loss: 2.6049 - val_acc: 0.5333
Epoch 755/1000
30/30 [==============================] - 0s - loss: 1.9452e-04 - acc: 1.0000 - val_loss: 2.6043 -

30/30 [==============================] - 0s - loss: 3.7696e-04 - acc: 1.0000 - val_loss: 2.6210 - val_acc: 0.5333
Epoch 812/1000
30/30 [==============================] - 0s - loss: 1.4233e-04 - acc: 1.0000 - val_loss: 2.6203 - val_acc: 0.5333
Epoch 813/1000
30/30 [==============================] - 0s - loss: 2.9247e-04 - acc: 1.0000 - val_loss: 2.6202 - val_acc: 0.5333
Epoch 814/1000
30/30 [==============================] - 0s - loss: 0.0013 - acc: 1.0000 - val_loss: 2.6226 - val_acc: 0.5333
Epoch 815/1000
30/30 [==============================] - 0s - loss: 4.1732e-04 - acc: 1.0000 - val_loss: 2.6236 - val_acc: 0.5333
Epoch 816/1000
30/30 [==============================] - 0s - loss: 7.8877e-05 - acc: 1.0000 - val_loss: 2.6235 - val_acc: 0.5333
Epoch 817/1000
30/30 [==============================] - 0s - loss: 7.2886e-04 - acc: 1.0000 - val_loss: 2.6254 - val_acc: 0.5333
Epoch 818/1000
30/30 [==============================] - 0s - loss: 1.3229e-04 - acc: 1.0000 - val_loss: 2.6246 - val

30/30 [==============================] - 0s - loss: 1.5466e-04 - acc: 1.0000 - val_loss: 2.5841 - val_acc: 0.5333
Epoch 875/1000
30/30 [==============================] - 0s - loss: 4.1407e-04 - acc: 1.0000 - val_loss: 2.5818 - val_acc: 0.5333
Epoch 876/1000
30/30 [==============================] - 0s - loss: 5.7064e-04 - acc: 1.0000 - val_loss: 2.5811 - val_acc: 0.5333
Epoch 877/1000
30/30 [==============================] - 0s - loss: 3.6535e-04 - acc: 1.0000 - val_loss: 2.5813 - val_acc: 0.5333
Epoch 878/1000
30/30 [==============================] - 0s - loss: 2.7914e-04 - acc: 1.0000 - val_loss: 2.5817 - val_acc: 0.5333
Epoch 879/1000
30/30 [==============================] - 0s - loss: 1.1394e-04 - acc: 1.0000 - val_loss: 2.5810 - val_acc: 0.5333
Epoch 880/1000
30/30 [==============================] - 0s - loss: 4.1957e-04 - acc: 1.0000 - val_loss: 2.5828 - val_acc: 0.5333
Epoch 881/1000
30/30 [==============================] - 0s - loss: 4.3162e-04 - acc: 1.0000 - val_loss: 2.5837 -

30/30 [==============================] - 0s - loss: 1.3482e-04 - acc: 1.0000 - val_loss: 2.6158 - val_acc: 0.5667
Epoch 938/1000
30/30 [==============================] - 0s - loss: 4.5493e-04 - acc: 1.0000 - val_loss: 2.6169 - val_acc: 0.5667
Epoch 939/1000
30/30 [==============================] - 0s - loss: 1.7957e-04 - acc: 1.0000 - val_loss: 2.6177 - val_acc: 0.5667
Epoch 940/1000
30/30 [==============================] - 0s - loss: 2.2109e-04 - acc: 1.0000 - val_loss: 2.6183 - val_acc: 0.5667
Epoch 941/1000
30/30 [==============================] - 0s - loss: 2.2857e-04 - acc: 1.0000 - val_loss: 2.6193 - val_acc: 0.5667
Epoch 942/1000
30/30 [==============================] - 0s - loss: 1.8090e-04 - acc: 1.0000 - val_loss: 2.6195 - val_acc: 0.5667
Epoch 943/1000
30/30 [==============================] - 0s - loss: 1.2880e-04 - acc: 1.0000 - val_loss: 2.6197 - val_acc: 0.5667
Epoch 944/1000
30/30 [==============================] - 0s - loss: 0.0014 - acc: 1.0000 - val_loss: 2.6235 - val

30/30 [==============================] - 0s - loss: 1.1133e-04 - acc: 1.0000 - val_loss: 2.6611 - val_acc: 0.5667
----------- Resultados -----------
Loss:  6.38791670402e-05
Acc:  0.533333333333
Tempo de processamento: 919.19 segundos.
